In [1]:
import pandas as pd
import itertools as it
import numpy as np

df = pd.read_excel(r"sample order basket.xlsx")

#df with SKU and collated orders in a list
def sorter(samplelist):
    samplelist.sort()
    return samplelist

def convert(list):
    return tuple(i for i in list)

def f(df):
    keys, values = df.sort_values('OrderNum').values.T
    ukeys, index = np.unique(keys, True)
    arrays = np.split(values, index[1:])
    df2 = pd.DataFrame({'OrderNum':ukeys, 'Product':[list(a) for a in arrays]})
    df2["Product"] = df2["Product"].apply(sorter)
    df2["Product"] = df2["Product"].apply(convert)
    return df2

df2 = f(df)

#Gets all products, and all combinations
product_list = list(df["Product"].unique())
product_list.sort()
ctr = 1

#product_dict is the name of all combinations... (item1,item2):"Grouping x"...
product_dict = {}

for i in range(2,len(product_list)+1):
    combinations = list(it.combinations(product_list,i))

    for grouping in combinations:
        nameOfKey = "Grouping "+str(ctr)
        product_dict[grouping] = nameOfKey
        ctr += 1


def categorizer(samplelist):
    if samplelist in product_dict:
        samplelist = product_dict[samplelist]
        return samplelist
    else:
        return samplelist
    
df2["Grouping"] = df2["Product"]
df2["Grouping"] = df2["Grouping"].apply(categorizer)


reverse_product_dict = {values:key for key,values in product_dict.items()}

def reverse_categorizer(samplelist):
    if samplelist in reverse_product_dict:
        samplelist=reverse_product_dict[samplelist]
        return samplelist
    else:
        return "Solo"
    
df2["Grouping 2"] = df2["Grouping"]
df2["Grouping 2"] = df2["Grouping 2"].apply(reverse_categorizer)

df_final = df2.copy()
df_final.drop(columns = ["OrderNum","Product"],inplace=True)
df_final = df_final.groupby("Grouping 2").count()
df_final.reset_index(inplace=True)
df_final.columns = ["Product Mix","Count"]
df_final

,Product Mix,Count
0,"(apple, banana)",1
1,"(apple, banana, carrot)",1
2,"(apple, banana, carrot, eggplant)",1
3,"(apple, eggplant)",2
4,"(carrot, eggplant)",1
5,Solo,5
